# INM432: Big Data - Coursework (Part II)

## Predicting shifts in GBP-EUR exchange rates based on the content of UK parliamentary debates: A pySpark application

### Alexandros Dimitrios Nalmpantis; Georgios Kyriakopoulos (2017)

## Study objective and approach 

The pySpark code presented in this notebook aims to construct a modelling process, which tests if shifts in GBP-EUR exchange rates can be predicted on the basis of political speech. More specifically, the code implements a process that involves the following steps:

(a) It scrapes the (almost daily) reports that record debates at the House of Commons and the House of Lords, as published on the UK Government website. The reports (available in PDF format) are converted into txt files and common (usually uninformative) words, numeric figures, and punctuation symbols are removed. File names and the remainder text content are converted into a data-frame and the content is tokenised and hashed, before computing term frequencies (TFs) and inverse document frequencies (IDFs) for the hashes.

(b) It scrapes the (almost daily) EUR-GBP exchange rates, published on the Bank of England website. The content is written in a text file, which is then processed to remove blank space and to calculate the difference between each GBP-EUR exchange-rate value and its immediately previous exchange-rate value (i.e. *the exchange-rate shift*). Dates and exchange-rate shifts are then converted into a second data-frame.

(c) It links the data frames constructed at (a) and (b) together, so that the content of the debate reports at a certain date is appended to the shift in GBP-EUR exchange rates recorded the day after. This new data-frame serves as the analytical input on the basis of which the main analysis is conducted.

(d) Finally, it constructs a machine learning pipeline, whereby the linear regression models are trained, validated, and tested to predict exchange-rate shifts based on either the TF or the IDF of the (tokenised and hashed) content of the debate reports. Alternative parameterisation options are explored using a grid.

The processes of data collection embedded in (a) and (b) depend on the format at which relevant data are published on the UK Government and the Bank of England websites. For this study, we collect data from 01/06/2016 onwards, and the functionality of the code that collects the data was last confirmed on 23/04/2017. However, future functionality  can be affected by future changes in the way that information is published. ***Along with this notebook, we provide the datafiles scraped on 23/04/2017, so that the analysis can be repeated when this code is used / assessed by others***.  

The findings of the analysis are highlighted throughout the annotation in this notebook. Overall (and perhaps unsurprisingly), the findings suggest that political speech at the House of Lords and the House of commons is unsuccessful in predicting complex and dynamic macroeconomic financial-market parameters, such as the GBP-EUR exchange-rate shifts. 

Given the time-series nature of the data that this analysis considers, we have (partially) addressed the issue of autocorrelations when using linear regression analysis by focusing the analysis on the prediction of exchange-rate shifts (rather than the prediction of exchange rate values). We suggest that the analysis is revisited in the future using perhaps more appropriate analytical techniques, such as Autoregressive Integrated Moving Average time-series models.


## 1 || Analysis modules

The modules needed for the analysis are imported below. Some modules may need to be installed to the terminal with the following commands: **pip install <"name of module">** eg: pip install tqdm  or **with conda install <"name of module">** eg: conda install tqdm.

The modules that will be required are **BeautifulSoup, urllib, re, wget, tika, tqdm, stop_words**.

The folder with the data and the notebook can be found in .zip format on:
https://cityuni-my.sharepoint.com/personal/alexandros_nalmpantis_city_ac_uk/_layouts/15/guestaccess.aspx?folderid=1f6589982e68d49499959d64f6a565018&authkey=AQW7rgXzZBcwHZW67Im0-P4

Alternatively the data can be scraped by selecting no to the following user inputs regarding the data. In this case the pachages listed above will be required.

In [2]:
# Modules used for scraping
from bs4 import BeautifulSoup
import urllib.request
import re
import datetime
from datetime import date,timedelta
import os

# Modules used for downloads
#import wget
import pandas as pd
import numpy as np

# Modules used for for parsing PDFs
import warnings
from tqdm import tnrange, tqdm_notebook
from tika import parser
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

# Spark ML and SQL modules
import re
from pyspark import SparkContext
from pyspark.ml.feature import Tokenizer,HashingTF, IDF
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder,TrainValidationSplit
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import *
from pyspark.sql import *

# Other modules
from math import log
import time
from pprint import pprint
import sys
import matplotlib.pyplot as plt
from  stop_words import get_stop_words
import warnings
warnings.filterwarnings("ignore")

# Set sparkcontext as sc
#sc=SparkContext() 

ModuleNotFoundError: No module named 'tqdm'

## 2 || Data collection and pre-processing

###     2.1 Procedures for scraping, downloading, and converting parliamentary-debate reports into text files

In [2]:
# Define if readily available debate-report data will be used (trg= 'yes') or if they will be scraped (trg ='no')
trg='yes' #change this value to 'yes' or 'no'

# Set date from which onwards debate-reports with be downloaded in YYYY-M-D format
start_date = date(2016, 6, 1)

In [3]:
# Data_control function that controls whether the readily available debate-report data will be used or if they will be scraped
def data_control(page,start_date,trg):
    if trg=='yes':
        print ('====Data were given====')
    else:
        html_page = urllib.request.urlopen(page) #request page with urllib packages
        soup = BeautifulSoup(html_page) #pass the page to beautiful soup in order to extract the links contained in webpage 
        #print (soup) #visually inspect the html structure
        hl = [] #set hyperlink array to store the extracted links
        ##search html for hyperlinks starting with qna
        for hyperlink in soup.findAll('a', attrs={'href': re.compile("^http://qna")}): 
            hl.append(hyperlink.get('href')) #store the hyperlinks found on an array
        #    print (link.get('href'))
        
        url=[hl[1][:-22]+'Lords-',hl[1][:-22]+'Commons-'] #take first result and cut the dates and category of either lords or commons
    
        ##create interval search date
        today=datetime.datetime.today() #today's date set
        cur_date = date(today.year,today.month,today.day)  # set current date in format of YYYY-MM-DD
    
        dt = cur_date - start_date #calculate interval in days to use for loop
        ##make directory to downloaded files
        try:
            os.makedirs(os.getcwd()+'/parliament_practicals') #make directory to downloaded files
        except:
            pass
    
        ##loop throught the interval with 1 day step and append the date to url along with categories of either House of Lords or Commons
        for ul in url:
        #    print ('Downloading: ',str('House of '+ul[112:-2]+'s'))
            for i in tnrange(dt.days + 1,desc='Downloading: '+str(ul[112:-2]+'s')):
                try: #test for errors and pass since there are dates that the House of Lords do not convene and HTTP request returns error; Also store results on folder parliament practicals
                    filename = wget.download(ul+str(start_date + timedelta(days=i))+'.pdf',os.getcwd()+'/parliament_practicals')
                except:
                    next  
    
# Function to convert the downloaded pdfs to text files
def convert_pdf_to_text(trg):
    if trg=='yes': # user input in case data are already given in appropriate format
        print ('====Data were given====')
    else:
        try: # test if directory textfiles already exists otherwise make the directory
            os.makedirs(os.getcwd()+'/textfiles') #make directory to downloaded files
        except:
            pass
        list_of_files=os.listdir(os.getcwd()+'/parliament_practicals') # create a list of pdf files to be converted
        for i in tnrange(len(list_of_files),desc='Converting pdf to txt'): # iterate throught the files on the list and install progress bar
            if list_of_files[i].endswith(".pdf"): # check that file input is pdf file
                parsedPDF=parser.from_file(os.getcwd()+'/parliament_practicals/'+list_of_files[i]) # parse pdf file
                text_file = open(os.getcwd()+'/textfiles/'+list_of_files[i][:-4]+'.txt', 'a') # create new filename with extension .txt
                text_file.write(parsedPDF["content"]) # write parsed pdf to text
                text_file.close() # close text file
            else: # if file other than pdf continue loop
                next
def download_practicals_convert(start_date,trg):
    page="http://www.parliament.uk/business/publications/written-questions-answers-statements/daily-reports/" # set link to parliament daily questions and answers reports
    data_control(page,start_date,trg) # call set data function
    convert_pdf_to_text(trg)# convert to pdf function
    print ('==Downloading and conversion to text files completed==')
    
# Call function to either download the data or set current folder as working folder...please make sure that
# if data are give then those should be stored on the folder: 'parliament_practicals'

download_practicals_convert(start_date,trg) 

### 2.2. Procedures for scraping, downloading, and converting exchange rates to exchange-rate shifts and storing in a data-frame

In [4]:
# Define if readily available exchange-rate data will be used (trg= 'yes') or if they will be scraped (trg ='no')
trg='yes' # change this value to yes or no

In [5]:
# Clean exchange data download function and transform to pandas dataframe

def clean_ex(file_path):
    data = pd.read_csv(file_path,sep=" t ",header=None, encoding="ISO-8859-1") # load the text file
    remove_words=['Bank of England Statistical Interactive Database','Series 1 to 1','Spot exchange rate, Euro into Sterling','XUDLERS','Ã‚','Â']
    for word in remove_words: # remove words
        data=data.replace(word,np.nan) # remove words
    data=data.dropna()# drop nan
    data.reset_index() # reset indices
    rate=data.iloc[::2] # extract odd rows
    date=data.iloc[1::2] # extract even rows
    date.reset_index(inplace=True,drop=True) # reset indices
    rate.reset_index(inplace=True,drop=True) # reset indices
    x=pd.concat([date,rate],axis=1) # concatenate date and rates
    x.columns=['Rate','Date'] # rename columns
    x=x.dropna()# drop na
    x['Date'] = pd.to_datetime(x['Date']) # convert date column to date
    x[['Rate']] = x[['Rate']].apply(pd.to_numeric) #convert exchange rate to float
    x.dtypes #check data types
    x = x.set_index('Date').diff() # calculate [rate(t+1) - rate(t)]
    x.reset_index(inplace=True)# reset Date column
    x['Date']=x['Date'].dt.strftime('%Y-%m-%d')# convert to string for join matching operations
    x.Rate = x.Rate.shift(-1)# shift rate column by one day to account for the delay of the parliament report
    x=x.dropna()# drop na
    x=x.drop_duplicates('Date') # drop duplicates
    x.to_csv(os.getcwd()+'/xr/exchangeRates_diff.csv')# save to csv file
    return x # return dataframe

# Define function to download exchange rates to text file in folder xr
def download_xr(trg):
    html= urllib.request.urlopen("http://www.bankofengland.co.uk/boeapps/iadb/fromshowcolumns.asp?Travel=NIxIRxSUx&FromSeries=1&ToSeries=50&DAT=RNG&FD=1&FM=Jan&FY=1963&TD=11&TM=Apr&TY=2017&VFD=Y&CSVF=TT&C=C8J&Filter=N&html.x=11&html.y=9")
    if trg=='yes': # user input in case data are already given in appropriate format
        print ('====Data were given====')
        data=clean_ex(os.getcwd()+'/xr/exchangeRates.txt')
    else:
        try: # test if directory xr already exists otherwise make the directory
            os.makedirs(os.getcwd()+'/xr') #make directory to downloaded files
        except:
            pass
        soup_xr = BeautifulSoup(html)
        xr = soup_xr.get_text()
        #print(xr)
        text_xr = open(os.getcwd()+'/xr/'+'exchangeRates'+'.txt', "a")
        text_xr.write(xr)
        text_xr.close()
        data=clean_ex(os.getcwd()+'/xr/exchangeRates.txt')
        next
    return data

data_xr = download_xr(trg)
print(data_xr.head()) # print head

====Data were given====
         Date    Rate
0  1999-01-04 -0.0019
1  1999-01-05  0.0082
2  1999-01-06  0.0007
3  1999-01-07  0.0068
4  1999-01-08  0.0011


## 3 || Defining machine-learning pipeline

### 3.1 Collate analysis dataset, run initial load, and perform transformations [Task A]

In [6]:
# Create dataframe of filename - text with numbers and punctuation removed

# Set stop word parameters-for stopwords removal the stop_words pachage was used
# The  StopWordsRemover from pyspark.ml.features was also tested extensively but was buggy / not effective
stop_words = get_stop_words('english')

def remove_n_p(text): # function that removes punctuation and numbers as well lowercasing the text
    text = re.sub(r'\d+','', text) # remove numbers from texts with regular expressions
    text = re.sub(r'\[.*?\]|\(.*?\)|\W+', ' ', text)# remove punctuation from texts with regular expressions
    text=text.lower() # lowercase the text
    text = ' '.join([word for word in text.split() if word not in stop_words]) # remove stopwords
    return text

# Extract date from filename function 
def trim_filename(filename):
    date=filename[-14:-4] # extract the timestamp from end of the file
    return date # return date
    
# SparkSession added for spark dataframes
spark = SparkSession.builder.getOrCreate()

def make_dataFrame(dirPath): # make a dataFrame with filename and text 
    ft_RDD = sc.wholeTextFiles(dirPath) # add code to create an RDD with wholeTextFiles
    spm_t_RDD = ft_RDD.map(lambda ft: (trim_filename(ft[0]), remove_n_p(ft[1]))) # create RDD with filename and call remove_n_p function to text
    file_text_df = spark.createDataFrame(spm_t_RDD,schema=['id','text']) # create a dataFrame - filename - text
    return file_text_df

In [20]:
# Convert currency pandas dataframe to Spark dataframe and specify datatypes

def currency_df(data_xr):# function to create dataframe
    data_xr_DF=spark.createDataFrame(data_xr,schema=['Date','Rate']) # create dataframe
    data_xr_DF=data_xr_DF.withColumn('Rate', data_xr_DF['Rate'].cast('float'))# convert rate to float
    return data_xr_DF # return spark dataframe

# function to join the two dataframes on dates. Exchange rate dates are shifted back by one day
# in order to account for the delay of the parliament publication
def connect_xr_df(file_text_df,data_xr_DF): # the function takes the two inpurts of file_text_df and the data_xr_DF from the previous function
    file_text_Date_rate=file_text_df.join(data_xr_DF,file_text_df.id==data_xr_DF.Date,'left_outer') # the exchange rates were connected to the timestamp of the parliament files with a matching left outer join
    file_text_Date_rate.createOrReplaceTempView("temp") # create a temporary sql view
    file_text_Date_rate_sql = spark.sql("SELECT id,text,Rate as label FROM temp") # select statement of the three columns required for analysis and relabeling
    file_text_Date_rate_sql.show(5)
    return file_text_Date_rate_sql # return the dataframe for analysis

id_text_label=connect_xr_df(make_dataFrame(os.getcwd()+'/textfiles'),currency_df(data_xr))

+----------+--------------------+-------+
|        id|                text|  label|
+----------+--------------------+-------+
|2017-02-24|daily report frid...|-0.0069|
|2017-02-24|friday february p...|-0.0069|
|2016-07-06|daily report wedn...| 0.0041|
|2016-07-06|wednesday july p ...| 0.0041|
|2017-03-08|daily report wedn...|-0.0044|
+----------+--------------------+-------+
only showing top 5 rows



### 3.2 Implement a machine-learning pipeline in Spark, including feature extractors, transformers, and selectors [Task B] 

In [21]:
#Step1: use tokenizer to split word into array and sql to select the filename - word_array created
tokenizer = Tokenizer(inputCol="text", outputCol="words")

#Step2: make hashTF 
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")

#Step 3: feed hash vector to calculate idf
idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol="idf") 

#Step4: linear regression parameters
lr_tf = LinearRegression()\
         .setFeaturesCol("features")\
     .setLabelCol('label')

lr_idf = LinearRegression()\
     .setFeaturesCol("idf")\
     .setLabelCol('label')

#Step 5: configure alternative pipelines 
pipeline_tf = Pipeline(stages=[tokenizer, hashingTF, lr_tf]) #with hash vector tf
pipeline_idf = Pipeline(stages=[tokenizer, hashingTF, idf, lr_idf]) #with hash vector idf 

#Step 6: set exemplar parameter grid
paramGrid_tf = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [100000]) \
    .addGrid(lr_tf.regParam, [0.3]) \
    .addGrid(lr_tf.maxIter, [50]) \
    .addGrid (lr_tf.elasticNetParam,[0.8])\
    .build()

paramGrid_idf = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [100000]) \
    .addGrid(lr_idf.regParam, [0.3]) \
    .addGrid(lr_idf.maxIter, [50]) \
    .addGrid (lr_idf.elasticNetParam,[0.8])\
    .build()

### 3.3 Evaluate pipeline performance using training and test datasets [Task C]

In [9]:
[id_text_label_train, id_text_label_test] = id_text_label.randomSplit([0.8, 0.2], 25) # split id_text_label into training (80%) and testing (20%) subsets, seed = 25

evaluator = RegressionEvaluator(metricName="r2", labelCol="label", predictionCol="prediction") # Use R squared to evaluate performance of models (% of variance in xr shifts explained by predictors)

### Modeling with pipeline_tf
tvs_tf = TrainValidationSplit(estimator=pipeline_tf, 
                           estimatorParamMaps=paramGrid_tf,
                           evaluator =evaluator,
                           trainRatio=0.8) # 80% of the data will be used for training, 20% for validation

# Run TrainValidationSplit on training dataset
print('starting Train-Validation')
tvsModel_tf = tvs_tf.fit(id_text_label_train)
print('finished Train-Validation')

# R squared for prediction on training dataframe 
prediction  = tvsModel_tf.transform(id_text_label_train)
print(tvsModel_tf.bestModel)
rsquared_tf = evaluator.evaluate(prediction)
print("---Linear Regression with hash TF predictors: R Squared for training dataset is %s ---" % (rsquared_tf))

# R squared for prediction on testing dataframe 
prediction  = tvsModel_tf.transform(id_text_label_test)
print(tvsModel_tf.bestModel)
rsquared_tf = evaluator.evaluate(prediction)
print("---Linear Regression with hash TF predictors: R Squared for testing dataset is %s ---" % (rsquared_tf))

### Modeling with pipeline pipeline_idf

tvs_idf = TrainValidationSplit(estimator=pipeline_idf, 
                           estimatorParamMaps=paramGrid_idf,
                           evaluator =evaluator,
                           trainRatio=0.8) # 80% of the data will be used for training, 20% for validation

# Run TrainValidationSplit, and choose the best set of parameters.
print('starting Train-Validation')
tvsModel_idf = tvs_idf.fit(id_text_label_train)
print('finished Train-Validation')

# R squared for prediction on training dataframe 
prediction  = tvsModel_idf.transform(id_text_label_train)
print(tvsModel_idf.bestModel)
rsquared_idf = evaluator.evaluate(prediction)
print("---Linear Regression with hash IDF predictors: R Squared for training dataset is %s ---" % (rsquared_idf))

# R squared for prediction on training dataframe 
prediction  = tvsModel_idf.transform(id_text_label_test)
print(tvsModel_idf.bestModel)
rsquared_idf = evaluator.evaluate(prediction)
print("---Linear Regression with hash IDF predictors: R Squared for testing dataset is %s ---" % (rsquared_idf))


starting Train-Validation
finished Train-Validation
PipelineModel_401398cbedea7e2a888d
---Linear Regression with hash TF predictors: R Squared for training dataset is 1.1102230246251565e-16 ---
PipelineModel_401398cbedea7e2a888d
---Linear Regression with hash TF predictors: R Squared for testing dataset is -0.05000512356683151 ---
starting Train-Validation
finished Train-Validation
PipelineModel_463cab1342fb343af0bc
---Linear Regression with hash IDF predictors: R Squared for training dataset is 1.1102230246251565e-16 ---
PipelineModel_463cab1342fb343af0bc
---Linear Regression with hash IDF predictors: R Squared for testing dataset is -0.05000512356683129 ---


### 3.4 Experiment with and evaluate alternative pipelines using grid [Task D]

In [23]:
#Set experiment parameter grid
paramGrid_tf_exp = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [100000, 200000]) \
    .addGrid(lr_tf.regParam, [0.1, 0.3]) \
    .addGrid(lr_tf.maxIter, [20, 50]) \
    .addGrid (lr_tf.elasticNetParam,[0.1, 0.8])\
    .build()

paramGrid_idf_exp = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [100000, 200000]) \
    .addGrid(lr_idf.regParam, [0.1, 0.3]) \
    .addGrid(lr_idf.maxIter, [20, 50]) \
    .addGrid (lr_idf.elasticNetParam,[0.1, 0.8])\
    .build()

[id_text_label_train, id_text_label_test] = id_text_label.randomSplit([0.8, 0.2], 25) # split id_text_label into training (80%) and testing (20%) subsets, seed = 25

evaluator = RegressionEvaluator(metricName="r2", labelCol="label", predictionCol="prediction") # Use R squared to evaluate performance of models (% of variance in xr shifts explained by predictors)

### Experiments with pipeline_tf

for ratio in [0.7, 0.8]:
    tvs_tf_exp = TrainValidationSplit(estimator=pipeline_tf,
                           estimatorParamMaps=paramGrid_tf_exp,
                           evaluator =evaluator,
                           trainRatio=ratio) # 80% of the data will be used for training, 20% for validation
    # Run TrainValidationSplit on training dataset
    print('starting Train-Validation with training to validation ratio = ('+str(ratio)+')')
    tvsModel_tf_exp = tvs_tf_exp.fit(id_text_label_train)
    print('finished Train-Validation')
    # R squared for prediction on training dataframe 
    prediction  = tvsModel_tf_exp.transform(id_text_label_train)
    print(tvsModel_tf_exp.bestModel)
    rsquared_tf_exp = evaluator.evaluate(prediction)
    print("---Linear Regression with hash TF predictors: R Squared for training dataset is %s ---" % (rsquared_tf_exp))
    # R squared for prediction on testing dataframe 
    prediction  = tvsModel_tf_exp.transform(id_text_label_test)
    print(tvsModel_tf_exp.bestModel)
    rsquared_tf_exp = evaluator.evaluate(prediction)
    print("---Linear Regression with hash TF predictors: R Squared for testing dataset is %s ---" % (rsquared_tf_exp))

### Experiments with pipeline pipeline_idf

for ratio in [0.7, 0.8]:
    tvs_idf_exp = TrainValidationSplit(estimator=pipeline_idf,
                           estimatorParamMaps=paramGrid_idf_exp,
                           evaluator =evaluator,
                           trainRatio=ratio) # 80% of the data will be used for training, 20% for validation
    # Run TrainValidationSplit on training dataset
    print('starting Train-Validation with training to validation ratio = ('+str(ratio)+')')
    tvsModel_idf_exp = tvs_idf_exp.fit(id_text_label_train)
    print('finished Train-Validation')
    # R squared for prediction on training dataframe 
    prediction  = tvsModel_idf_exp.transform(id_text_label_train)
    print(tvsModel_idf_exp.bestModel)
    rsquared_idf_exp = evaluator.evaluate(prediction)
    print("---Linear Regression with hash IDF predictors: R Squared for training dataset is %s ---" % (rsquared_idf_exp))
    # R squared for prediction on testing dataframe 
    prediction  = tvsModel_idf_exp.transform(id_text_label_test)
    print(tvsModel_idf_exp.bestModel)
    rsquared_idf_exp = evaluator.evaluate(prediction)
    print("---Linear Regression with hash IDF predictors: R Squared for testing dataset is %s ---" % (rsquared_idf_exp))




starting Train-Validation with training to validation ratio = (0.7)
finished Train-Validation
PipelineModel_4aed8d38e362b986c773
---Linear Regression with hash TF predictors: R Squared for training dataset is 0.0 ---
PipelineModel_4aed8d38e362b986c773
---Linear Regression with hash TF predictors: R Squared for testing dataset is -0.05000512356683151 ---
starting Train-Validation with training to validation ratio = (0.8)
finished Train-Validation
PipelineModel_441794d9653cd168b660
---Linear Regression with hash TF predictors: R Squared for training dataset is -2.220446049250313e-16 ---
PipelineModel_441794d9653cd168b660
---Linear Regression with hash TF predictors: R Squared for testing dataset is -0.05000512356683129 ---
starting Train-Validation with training to validation ratio = (0.7)
finished Train-Validation
PipelineModel_4a42b7461d273edff98e
---Linear Regression with hash IDF predictors: R Squared for training dataset is 0.0 ---
PipelineModel_4a42b7461d273edff98e
---Linear Regres

## 3 || Comments / summary of findings

The results of this preliminary analysis suggest that the content of political speech at the House of Commons and the House of Lords does not account for the variation in shifts of GBP-EUR exchange rates, at least to a degree that is detectible by the linear regression models that were trained, validated, and tested in this study. The proportion of variation in exchange-rate shifts predicted by either term frequencies or inverse document frequencies (as indicated by R squared metrics presented in this notebook) at the training stage of the linear regression algorithm is c.0 for alternative linear regression parameters. The experiments conducted suggest that increasing the number of hashes (thus reducing collisions during text processing) does not improve results. R squared metrics yielded at the testing phase of the algorithms are systematically marginally below 0. This is indicative of poor predictions on the testing datasets; poorer than predictions at random. Overall, findings are probably non-exciting but certainly non counter-intuitive, since the outcome we attempted to predict is complex, dynamic, and dependent upon highly complex market processes. 

In [4]:
from pyspark import SparkContext


In [8]:
SparkContext


pyspark.context.SparkContext